# Question 1

In this section we will create a dataframe from the Wikipedia information regarding neighborhoods in Toronto

In [13]:
#import what we'll need for this project
import pandas as pd
import numpy

In [3]:
#scrape the information from the Wikipedia page
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
tables = pd.read_html(url)
tables[0].head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [22]:
#place the data into a DataFrame
df1=pd.DataFrame(tables[0])
df1.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [23]:
#create a second DataFrame with the null value entries removed from the Borough column
df2 = df1[df1.Borough != 'Not assigned']
df2.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [24]:
#create a third DataFrame with the neighbourhoods combined under duplicate postcodes
df3=df2.groupby(['Postcode','Borough'],as_index=False, sort=False).agg(','.join)
df3.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Not assigned


In [25]:
#replace the null values in the Neighbourhood column with the Borough column value
df3.Neighbourhood.replace('Not assigned',df3.Borough,inplace=True)
df3.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [10]:
#return the number of rows and columns
df3.shape

(103, 3)

# Question 2

In this section we will find the latitude and longitude of each neighborhood

In [15]:
#import the geospatial data
table1=pd.read_csv('http://cocl.us/Geospatial_data')

In [16]:
#create a DataFrame with the geospatial data
df4=pd.DataFrame(table1)
df4.head()

In [19]:
#edit the column headings to be compatible with the Question 1 DataFrame
df4.columns=['Postcode','Latitude','Longitude']

In [26]:
#merge the two DataFrames
df5 = pd.merge(df3, df4, how='inner', on = 'Postcode')
df5.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
